## *जातस्य हि ध्रुवो मृत्युर्ध्रुवं जन्म मृतस्य च |<br>तस्मादपरिहार्येऽर्थे न त्वं शोचितुमर्हसि ||*

*Death is certain for one who has been born, and rebirth is inevitable for one who has died.<br>Therefore, you should not lament over the universally inevitable.*

*\- Bhagwat Gita Verse 2.27*

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm #colormap
import seaborn as sns

sns.set_style("dark")

from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import geopandas as gpd

In [ ]:
sdata = pd.read_csv('../input/suicides-in-india/Suicides in India 2001-2012.csv')
india = gpd.read_file('../input/india-states/Igismap/Indian_States.shp')

In [ ]:
sdata.head(10)

In [ ]:
sdata = sdata.drop(sdata[sdata.Total==0].index)

print("After Total = 0 is dropped",sdata['Type_code'].unique())

sdata = sdata.drop(sdata[(sdata['State'] == 'Total (Uts)') | (sdata['State'] == 'Total (All India)') | 
               (sdata['State'] == 'Total (States)') | (sdata['State'] == 'D & N Haveli') ].index)

print("After Amb. States are dropped",sdata['Type_code'].unique())

sdata = sdata.drop(sdata[(sdata['Age_group'] == '0-100+')].index)

print("After Age Group Amb is dropped",sdata['Type_code'].unique())

In [ ]:
sdata['State'].replace({'A & N Islands':'Andaman & Nicobar Island',
                        'Delhi (Ut)':'NCT of Delhi',
                       }, inplace = True)

india.rename(columns = {'st_nm':'State'}, inplace = True)
india['State'].replace({'Telangana':'Andhra Pradesh'
                       }, inplace = True)

<a id="table-of-contents"></a>
# Overview of Notebook

1. [Yearwise Overview](#yearwise)
1. [Major Geospatial Analysis Overview](#geo)
1. [Age](#age)
1. [Gender](#gender)
1. [Occuptation](#occupation)
1. [Comparision](#comparision)

<a id="yearwise"></a>
[Return to table of contents](#table-of-contents)
# 1. Yearwise Overview

In [ ]:
trace1 = go.Bar(
    x = sdata['Year'].unique(),
    y = sdata.groupby('Year').sum()['Total']
)

layout = go.Layout()
fig = go.Figure(data = trace1, layout = layout)

fig.update_layout(
    title="Yearwise suicide",
    xaxis_title="Year",
    yaxis_title="Suicide Count",
)
fig.update_yaxes(tickangle=-30, tickfont=dict(size=7.5))

iplot(fig)

<a id="geo"></a>
[Return to table of contents](#table-of-contents)
# 2. Geospatial Analysis

In [ ]:
sdata_states = sdata.groupby(['State']).agg({'Total':'sum','Year':'count'})
sdata_states.rename(columns = {'Year':'Cases Type Count'}, inplace = True)

### 2.1 Statewise Suicide Mapping

In [ ]:
suicide_map = india.merge(sdata_states, left_on='State', right_on='State')

suicide_map['coords'] = suicide_map['geometry'].apply(lambda x: x.representative_point().coords[:])
suicide_map['coords'] = [coords[0] for coords in suicide_map['coords']]

fig, ax = plt.subplots(figsize=(22, 15))

cmap = 'Reds'

ax = suicide_map.plot(ax=ax, cmap=cmap,column = 'Total',scheme = 'equal_interval',edgecolor = 'black')
ax.set_facecolor('white')
ax.set_title('Suicide Cases per State')

for idx, row in suicide_map.iterrows():
   ax.text(row.coords[0], row.coords[1], s=row['Total'], 
           horizontalalignment='center', bbox={'facecolor': 'white', 'alpha':0.8, 'pad': 2, 'edgecolor':'none'})

norm = matplotlib.colors.Normalize(vmin=suicide_map['Total'].min(), vmax= suicide_map['Total'].max())
n_cmap = cm.ScalarMappable(cmap= cmap, norm = norm)
n_cmap.set_array([])
ax.get_figure().colorbar(n_cmap)

#suicide_map[suicide_map['Total'] > 0].plot(ax=ax, cmap=cmap, markersize=1)

plt.xticks([])
plt.yticks([])
plt.show()

### 2.2 Barplot of Statewise Suicide Cases

In [ ]:
trace1 = go.Bar(
    x = sdata_states.sort_values(by = ['Total'],ascending = False).index,
    y = sdata_states.sort_values(by = ['Total'],ascending = False)['Total']
)

layout = go.Layout()
fig = go.Figure(data = trace1, layout = layout)

fig.update_layout(
    title="Statewise Suicide Count",
    xaxis_title="State",
    yaxis_title="Suicide Count",
)

iplot(fig)

### Observations

- It's quite surprising to see the South End of India having the highest majority of suicide rates. In the East part, West Bengal seems to be an exception to low suicide rates.
- However, these are purely based on numbers, so it's not uncommon to see higher populated areas registering more suicide rates compared to other areas.
- Population density is fairly low in North East states, and hence there is a decline in suicide rates towards the North East. The same argument can be applied for Lakshadweep, Andaman & Nicobar and Maldive Islands

<a id="age"></a>
[Return to table of contents](#table-of-contents)
# 3. Age

### 3.1 Age-Group Distribution of Suicide Cases

In [ ]:
sdata_age = sdata.groupby(['Age_group','Year']).sum()

In [ ]:
import plotly.express as px

df = px.data.tips()
fig = px.histogram(sdata, x="Age_group", y='Total',
                   color_discrete_sequence=['indianred','lightblue'],
                   marginal="violin", # or violin, rug
                  )

fig.update_layout(
    title="Suicide Cause",
    xaxis_title="Age Group",
    yaxis_title="Total Suicide Count",
)

fig.show()

### 3.2 Year-wise distribution of Age Groups

In [ ]:
import plotly.express as px

df = px.data.tips()
fig = px.histogram(sdata, y="Age_group", x='Total',color = 'Year',
                   color_discrete_sequence = px.colors.qualitative.G10,
                   marginal=None, # or violin, rug
                  )

fig.update_layout(
    title="Suicide for Age Group, Year-wise",
    xaxis_title="Total Suicide Count",
    yaxis_title="Age Group",
)
fig.update_yaxes(tickangle=-30, tickfont=dict(size=7.5))

fig.show()

### 3.3 Geospatial Analysis (Young vs Elder)

In [ ]:
sdata_agegroup = sdata.groupby(['Age_group','State']).sum()
sdata_agegroup

#### 3.3.1 Young Age Group State-wise Mapping

In [ ]:
each_age_range = sdata_agegroup.shape[0]//sdata['Age_group'].nunique()

suicide_map_younger = india.merge(sdata_agegroup[:each_age_range*2], left_on= 'State', right_on = 'State')
suicide_map_elder = india.merge(sdata_agegroup[each_age_range*2:], left_on = 'State', right_on = 'State')

fig, ax = plt.subplots(figsize=(22, 15))

cmap = 'Blues'

ax = suicide_map_younger.plot(ax=ax, cmap=cmap,column = 'Total',scheme = 'equal_interval',edgecolor = 'black')
ax.set_facecolor('white')
ax.set_title('Suicide Cases per State (Young)')

suicide_map_younger['coords'] = suicide_map_younger['geometry'].apply(lambda x: x.representative_point().coords[:])
suicide_map_younger['coords'] = [coords[0] for coords in suicide_map_younger['coords']]

for idx, row in suicide_map_younger.iterrows():
   ax.text(row.coords[0], row.coords[1], s=row['Total'], 
           horizontalalignment='center', bbox={'facecolor': 'white', 'alpha':0.8, 'pad': 2, 'edgecolor':'none'})


norm = matplotlib.colors.Normalize(vmin=suicide_map_younger['Total'].min(), vmax= suicide_map_younger['Total'].max())
n_cmap = cm.ScalarMappable(cmap= cmap, norm = norm)
n_cmap.set_array([])
ax.get_figure().colorbar(n_cmap)

plt.xticks([])
plt.yticks([])
plt.show()

#### 3.3.2 Elder Age Group State-wise Mapping

In [ ]:
fig, ax = plt.subplots(figsize=(22, 15))

cmap = 'binary'

ax = suicide_map_elder.plot(ax=ax, cmap=cmap,column = 'Total',scheme = 'equal_interval',edgecolor = 'black')
ax.set_facecolor('white')
ax.set_title('Suicide Cases per State (Elder)')

suicide_map_elder['coords'] = suicide_map_elder['geometry'].apply(lambda x: x.representative_point().coords[:])
suicide_map_elder['coords'] = [coords[0] for coords in suicide_map_elder['coords']]

for idx, row in suicide_map_elder.iterrows():
   ax.text(row.coords[0], row.coords[1], s=row['Total'], 
           horizontalalignment='center', bbox={'facecolor': 'white', 'alpha':0.8, 'pad': 2, 'edgecolor':'none'})

norm = matplotlib.colors.Normalize(vmin=suicide_map_elder['Total'].min(), vmax= suicide_map_elder['Total'].max())
n_cmap = cm.ScalarMappable(cmap= cmap, norm = norm)
n_cmap.set_array([])
ax.get_figure().colorbar(n_cmap)

plt.xticks([])
plt.yticks([])
plt.show()

<a id="gender"></a>
[Return to table of contents](#table-of-contents)
# 4. Gender

### 4.1 Barplot by Gender

In [ ]:
import plotly.express as px

df = px.data.tips()
fig = px.histogram(sdata, x="Gender", y='Total',color = 'Gender',
                   color_discrete_sequence=['indianred','lightblue'],
                   marginal=None, # or violin, rug
                  )

fig.update_layout(
    title="Gender-wise Distinction",
    xaxis_title="Gender",
    yaxis_title="Total count",
)

fig.show()

In [ ]:
sdata_sex = sdata.groupby(['Gender','State']).sum()

### 4.3 State-wise Suicide Mapping (Female)

In [ ]:
suicide_map_female = india.merge(sdata_sex[:sdata_sex.shape[0]//2], left_on= 'State', right_on = 'State')
suicide_map_male = india.merge(sdata_sex[sdata_sex.shape[0]//2:], left_on = 'State', right_on = 'State')

fig, ax = plt.subplots(figsize=(22, 15))

cmap = 'PuRd'

ax = suicide_map_female.plot(ax=ax, cmap=cmap,column = 'Total',scheme = 'equal_interval',edgecolor = 'black')
ax.set_facecolor('white')
ax.set_title('Suicide Cases per State (Female)')

suicide_map_female['coords'] = suicide_map_female['geometry'].apply(lambda x: x.representative_point().coords[:])
suicide_map_female['coords'] = [coords[0] for coords in suicide_map_female['coords']]

for idx, row in suicide_map_female.iterrows():
   ax.text(row.coords[0], row.coords[1], s=row['Total'], 
           horizontalalignment='center', bbox={'facecolor': 'white', 'alpha':0.8, 'pad': 2, 'edgecolor':'none'})

norm = matplotlib.colors.Normalize(vmin=suicide_map_female['Total'].min(), vmax= suicide_map_female['Total'].max())
n_cmap = cm.ScalarMappable(cmap= cmap, norm = norm)
n_cmap.set_array([])
ax.get_figure().colorbar(n_cmap)

plt.xticks([])
plt.yticks([])
plt.show()

### 4.4 State-wise Suicide Mapping (Male)

In [ ]:
fig, ax = plt.subplots(figsize=(22, 15))

cmap = 'Blues'

ax = suicide_map_male.plot(ax=ax, cmap=cmap,column = 'Total',scheme = 'equal_interval',edgecolor = 'black')
ax.set_facecolor('white')
ax.set_title('Suicide Cases per State (Male)')

suicide_map_male['coords'] = suicide_map_male['geometry'].apply(lambda x: x.representative_point().coords[:])
suicide_map_male['coords'] = [coords[0] for coords in suicide_map_male['coords']]

for idx, row in suicide_map_male.iterrows():
   ax.text(row.coords[0], row.coords[1], s=row['Total'], 
           horizontalalignment='center', bbox={'facecolor': 'white', 'alpha':0.8, 'pad': 2, 'edgecolor':'none'})

norm = matplotlib.colors.Normalize(vmin=suicide_map_male['Total'].min(), vmax= suicide_map_male['Total'].max())
n_cmap = cm.ScalarMappable(cmap= cmap, norm = norm)
n_cmap.set_array([])
ax.get_figure().colorbar(n_cmap)

plt.xticks([])
plt.yticks([])
plt.show()

<a id="occupation"></a>
[Return to table of contents](#table-of-contents)
# 5. Occupation

### 5.1 Major Breakdown - Generic Causes, Adoption and Professional Reasons

In [ ]:
sdata_type = sdata.groupby(['Type_code','Year']).sum()
dist_df = sdata_type.reset_index(level=[0,1]) #Multi-index to Single-index

trace0 = go.Box(y=dist_df.loc[dist_df['Type_code']=='Causes']['Total'],name="Causes")
trace1 = go.Box(y=dist_df.loc[dist_df['Type_code']=='Means_adopted']['Total'],name="Adoption")
trace2 = go.Box(y=dist_df.loc[dist_df['Type_code']=='Professional_Profile']['Total'],name="Professional")
data = [trace0,trace1,trace2]

#fig = px.box(dist_df, y='Year', x="Total", points="all", color = 'Type_code')
iplot(data)

### 5.2 By Minor Labels - Indepth Causes (Minor Types)

#### 5.2.1 All Minor Labels

In [ ]:
sdata['Type'].unique()

In [ ]:
sdata['Type'].replace({'Others (Please Specify)':'Unspecified',
                      'Causes Not Known':'Unspecified',
                       'Causes Not known':'Unspecified',
                       'Other Causes (Please Specity)':'Unspecified',
                       'By Other means (please specify)':'Unspecified',
                       'By Other means' : 'Unspecified',
                       'Unemployment':'Unemployed'
                      },inplace = True)

In [ ]:
fig = px.histogram(sdata, x="Total", y="Type", color = 'Gender',
                   marginal="violin", # or violin, rug,
                   color_discrete_sequence=['indianred','lightblue'],
                   )

fig.update_layout(
    title="Suicide Cause",
    xaxis_title="Total Suicide Count",
    yaxis_title="Suicide Cause/Method",
)
fig.update_yaxes(tickangle=-30, tickfont=dict(size=7.5))

fig.show()

#### 5.2.2 Crosschecking Numbers - Unspecified count

In [ ]:
#From Above chart, Unspecified stands at 759k + 307k = 1066k cases of suicide approx

sdata.loc[sdata['Type']=='Unspecified']['Total'].sum()/1000

#### 5.2.3 Top 20 Minor Labels (for cumulative Suicide count of both Genders)

In [ ]:
temp_df = sdata.groupby('Type').sum()['Total']
temp_df = temp_df.reset_index()
temp_df = temp_df.sort_values(by = ['Total'],ascending = False)
temp_df = temp_df[:20]
temp_df.rename(columns = {'Type':'Type Minor',
                          'Total':'Total Cases'},inplace = True)

In [ ]:
temp_df = temp_df.reset_index()
temp_df.drop(columns = ['index'],inplace = True)

In [ ]:
hist_dict = dict(zip(temp_df['Type Minor'], temp_df['Total Cases']))
hist_dict.keys()

In [ ]:
to_select = ['Unspecified', 'By Hanging', 'Family Problems', 'House Wife', 'By Consuming Insecticides', 'By Consuming Other Poison',
             'Farming/Agriculture Activity', 'Other Prolonged Illness', 'Unemployed', 'By Fire/Self Immolation', 'Service (Private)', 
             'By Drowning', 'Insanity/Mental Illness', 'Self-employed (Business activity)', 'Student', 'By coming under running vehicles/trains',
             'Love Affairs', 'Professional Activity', 'Bankruptcy or Sudden change in Economic', 'Poverty']

super_temp = sdata.loc[sdata['Type'].isin(to_select)]

In [ ]:
i = 10
for key in hist_dict:
    
    hist_dict[key] = str(i)+" "+key
    i +=1

In [ ]:
super_temp['Type'].replace(hist_dict, inplace = True)

In [ ]:
super_temp = super_temp.sort_values(by = ['Type'])

In [ ]:
fig = px.histogram(super_temp, x="Total", y="Type", color = 'Gender',
                   marginal='box', # or violin, rug,
                   color_discrete_sequence=['lightblue','indianred'],
                   ).update_yaxes(categoryorder = 'category descending')

fig.update_layout(
    title="Suicide Cause",
    xaxis_title="Total Suicide Count",
    yaxis_title="Suicide Cause/Method",
)
fig.update_yaxes(tickangle=-30, tickfont=dict(size=7.5))

fig.show()

##### Offtopic Explanation

The reason every entry in the plot above supports a number behind it is because every label in yaxis (cause of death) is not a continuous numerical variable.

- plotly doesn't have any convinient method as of yet to sort the axes, except for numerical values.
- A sorting done on categorical labels on any axis (through `category descending`) just orders the labels through alphabetical order.
- A workaround for this was to sort using `value_counts(ascending = False)` and mapping the names by adding a number starting from 10 onwards as a string with the top 20 results in order.
- Numbering starts from 10 because numbers like 3,4,5,6 etc come **after** 11,12,13 and so on wehen sorted by alphabetical order. Thus, the initial chain is broken if not 10.

#### 5.2.4 Top 20 Minor Lables (excluding 'Unspecified')

In [ ]:
super_temp = super_temp.drop(super_temp[(super_temp['Type'] == '10 Unspecified')].index)

fig = px.histogram(super_temp, x="Total", y="Type", color = 'Gender',
                   marginal='violin', # or violin, rug,
                   color_discrete_sequence=['lightblue','indianred'],
                   ).update_yaxes(categoryorder = 'category descending')

fig.update_layout(
    title="Suicide Cause",
    xaxis_title="Total Suicide Count",
    yaxis_title="Suicide Cause/Method",
)
fig.update_yaxes(tickangle=-30, tickfont=dict(size=7.5))

fig.show()

<a id="comparisions"></a>
[Return to table of contents](#table-of-contents)
# 6. Comparisions

### 6.1 Age Groups vs Year vs Gender vs Suicide Count

In [ ]:
fig = px.scatter_3d(sdata,x="Year",y="Age_group",z="Total",
    color = 'Gender', size_max = 18,
    color_discrete_sequence=['indianred','lightblue']
    )

fig.show()

### 6.2 Age and Gender vs Suicide Count

In [ ]:
import plotly.express as px

fig = px.histogram(sdata, x="Age_group", y="Total", color="Gender",
                   color_discrete_sequence=['indianred','lightblue'],
                   marginal="violin", # or violin, rug
                  )

fig.update_layout(
    xaxis_title="Age Group",
    yaxis_title="Suicide count",
)
fig.update_yaxes(tickangle=-30, tickfont=dict(size=7.5))

fig.show()

### 6.3 Suicide distribution by Major Occupation/Causes Labels

In [ ]:
super_temp2 = sdata.groupby(['Type_code','Gender']).sum()
super_temp2 = super_temp2.reset_index()
super_temp2 = super_temp2.sort_values(by = ['Gender'])

girl_occupation_suicide = super_temp2.iloc[:3]
boy_occupation_suicide = super_temp2.iloc[3:]

We won't be needing the `Year`, but since we have the options to pass argument's we'll leave this dataframe as is.

In [ ]:
import plotly.graph_objects as go

occupations_major = ['Causes','Means_adopted','Professional Profile']
female_death = list(super_temp2['Total'][:3])
male_death = list(super_temp2['Total'][3:])

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=male_death,
      theta=occupations_major,
      fill='toself',
      name='Male',
    line_color ="lightblue"
))

fig.add_trace(go.Scatterpolar(
      r=female_death,
      theta=occupations_major,
      fill='toself',
      name='Female',
    line_color ="indianred"
))

#fig = px.line_polar(boy_occupation_suicide, r='Total', theta='Type_code', line_close=True)
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0,1000000]
    )),
  showlegend=True
)

fig.update_traces(fill='toself')
fig.show()

### 6.4 Suicide distribution by Major Occupation/Causes Labels

In [ ]:
indexes = sdata.groupby('Type').sum()
indexes.sort_values(by = ['Total'], ascending = False)[1:11].index

In [ ]:
super_temp3 = sdata.groupby(['Gender','Type']).sum()
super_temp3 = super_temp3.reset_index()

In [ ]:
to_choose = ['By Hanging', 'Family Problems', 'House Wife',
       'By Consuming Insecticides', 'By Consuming Other Poison',
       'Farming/Agriculture Activity', 'Other Prolonged Illness', 'Unemployed',
       'By Fire/Self Immolation', 'Service (Private)']

super_temp3_female = super_temp3.loc[ (super_temp3['Type'].isin(to_choose)) & (super_temp3['Gender']=='Female')]
super_temp3_male = super_temp3.loc[ (super_temp3['Type'].isin(to_choose)) & (super_temp3['Gender']=='Male')]

super_temp3_male = super_temp3_male.append({'Type':'House Wife',
                        'Gender':'Male',
                        'Year':0,
                        'Total':0},ignore_index = True)

super_temp3_male = super_temp3_male.sort_values(by = ['Type'])
super_temp3_female = super_temp3_female.sort_values(by = ['Type'])

In [ ]:
import plotly.graph_objects as go

#female_death_minor = list(super_temp3_female['Total'])
#male_death_minor = list(super_temp3_male['Total'])

female_death_minor = [94703, 83569, 84316, 148262, 133077, 27300, 285021, 69052, 18608, 24289]
male_death_minor = [180772, 147483, 43656, 312223, 208688, 170455, 0, 125454, 96737, 117408]

#theta = list(super_temp3_female['Type'])

theta = ['By Consuming Insecticides',
 'By Consuming Other Poison',
 'By Fire/Self Immolation',
 'By Hanging',
 'Family Problems',
 'Farming/Agriculture Activity',
 'House Wife',
 'Other Prolonged Illness',
 'Service (Private)',
 'Unemployed']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=male_death_minor,
      theta=theta,
      fill='toself',
      name='Male',
      line_color ="lightblue"
))

fig.add_trace(go.Scatterpolar(
      r=female_death_minor,
      theta=theta,
      fill='toself',
      name='Female',
      line_color ="indianred"
))

#fig = px.line_polar(boy_occupation_suicide, r='Total', theta='Type_code', line_close=True)
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0,322223]
    )),
  showlegend=True
)

fig.update_traces(fill='toself')
fig.show()

### Observations

- As expected women have the most suicides cases originating/ tracing causes to being a housewife. The same stat scores a 0 for male suicide count for obvious reasons.
- Apart from the house-wife occupation, the only cause that has more cases of female suicide case is caused by fire/ immolation. Immolation (when something is killed or offered as a sacrifice) might trace it's roots back to the practice of [Sati](https://en.wikipedia.org/wiki/Sati_(practice)), though the practice has been banned and is rarely seen nowadays by official resources.
- The most common cause of death amongst men seem to be caused by hanging.
- Apart from suicide by fire and being a house-wife, men have a higher case of suicide in all the other top causes of a suicide in India.

# Placard

- There will be no comparision of suicide on basis of Gender using Placards as Gender is not a moral factor of estimating which suicide has more weightage. 
- We'll compare other factors for this Placard Summary. 
- Gender placards will be divided in two separate sections for comparing suicides grouped by respective gender, not by comparision between them.

### Cummulative Suicide Placard

In [ ]:
col = ['Total Suicide\n Count', 'Most Common\n Cause', 'Most affected \nState', 'Most affected\n Age Group',]

values = [sdata.groupby('Gender').sum()['Total']['Female'] + sdata.groupby('Gender').sum()['Total']['Male'],'By Hanging','Maharashtra','15-29']
color_val = ['lightblue','lightblue','lightblue','lightblue']

fig, axes = plt.subplots(1, 4, figsize=(24, 4))
axes = axes.flatten()
fig.set_facecolor('white')

for ind, col in enumerate(col):
    axes[ind].text(0.5, 0.6, col, 
            ha='center', va='center',
            fontfamily='monospace', fontsize=32,
            color='white', backgroundcolor='#8A0303')

    axes[ind].text(0.5, 0.2, values[ind], 
            ha='center', va='center',
            fontfamily='monospace', fontsize=38, fontweight='bold',
            color='#660000', backgroundcolor='white')
    
    axes[ind].set_axis_off()

### Male Suicide Placard

In [ ]:
col = ['Suicide Count', 'Most Common\n Cause', 'Most affected \nState', 'Most affected\n Age Group',]

values = [sdata.groupby('Gender').sum()['Total']['Male'],'By Hanging','Maharashtra','30-44']

fig, axes = plt.subplots(1, 4, figsize=(24, 4))
axes = axes.flatten()
fig.set_facecolor('white')

for ind, col in enumerate(col):
    axes[ind].text(0.5, 0.6, col, 
            ha='center', va='center',
            fontfamily='monospace', fontsize=32,
            color='white', backgroundcolor='#2D383A')

    axes[ind].text(0.5, 0.2, values[ind], 
            ha='center', va='center',
            fontfamily='monospace', fontsize=38, fontweight='bold',
            color='#2887C8', backgroundcolor='white')
    
    axes[ind].set_axis_off()

### Female Suicide Placard

In [ ]:
col = ['Suicide Count', 'Most Common\n Cause', 'Most affected \nState', 'Most affected\n Age Group',]

values = [sdata.groupby('Gender').sum()['Total']['Female'],'Being Housewife','West\n Bengal','15-29']

fig, axes = plt.subplots(1, 4, figsize=(24, 4))
axes = axes.flatten()
fig.set_facecolor('white')

for ind, col in enumerate(col):
    axes[ind].text(0.5, 0.6, col, 
            ha='center', va='center',
            fontfamily='monospace', fontsize=32,
            color='white', backgroundcolor='#2D383A')

    axes[ind].text(0.5, 0.2, values[ind], 
            ha='center', va='center',
            fontfamily='monospace', fontsize=38, fontweight='bold',
            color='indianred', backgroundcolor='white')
    
    axes[ind].set_axis_off()

# Offtopic : Homage to a College Alumni

- I was not planning on writing another kernel for a week or so. <br>
- However, some unfortune turns of events lead to a tragic event with one of my college's alumni. 
- I decided to write this kernel in my free time, to explore the ugly topic of depression, which I sadly have to admit is not taken as a serious state of someone's wellbeing in my country; and it's upto a point where suicides have been normalized in the society.

[Sushant Singh Rajput](https://www.google.com/search?gs_ssp=eJzj4tLP1TdISqrIyS4zYPQSKS4tzkjMK1EozsxLz1AoSswqKC0BAMj2DB0&q=sushant+singh+rajput&rlz=1C1SQJL_enIN886IN886&oq=sus&aqs=chrome.1.69i57j46j69i59l3j69i60l3.2247j0j7&sourceid=chrome&ie=UTF-8) enrolled in Dehli Technological University (then Delhi College of Engineering) in 2003. He would eventually drop out of the college after 3 years to pursue his dream of becoming an actor. <br>
Unlike many priviledged names in the Bollywood industry, he made a name for himself on his own merit and hardwork, eventually acting in movies depicting life of [Indian Cricketer MS Dhoni](https://en.wikipedia.org/wiki/M.S._Dhoni:_The_Untold_Story), Kai-po-che and a highly under-rated movie Chhichhore (Hindi word - meaning 'Shallow'), showcasing the vibrant college life along with topics of overwhelming anxiety, thoughts of depression and battling suicidal tendencies in teens; while spending his free time donating, helping others and fulfilling [his bucket list](https://economictimes.indiatimes.com/magazines/panache/sushant-singh-rajput-ticked-12-of-50-goals-off-his-bucket-list-actor-was-passionate-about-science-astronomy/articleshow/76384361.cms).
<br><br>
![](https://i.imgur.com/zW1h00J.png)
<br><br>
I remember him taking time out of his busy schedule, to visit his college on a Sunday, which ironically was a day of our Final Exams for a semester. The crowd was pretty pumped up to say the least. He walked in the library, hugged the students, the guards and the caterer of the college canteen for giving him a meal. Children chased his car as he departed out of the college gates.
<br><br>
![](https://i.imgur.com/uPdqvxW.png)
<br><br>
Sadly, most of what he did was left unnoticed by majority of public as they were not "spicy news" to be featured on the front lines, or gossip blogs and he continued to do what he believed in. This is very evident by his twitter where he kept posting of his bucket lists to keep him occupied, and instagram where he used to sometimes call out to people to watch his movies, as he had no "Godfather" to take shelter under for his acting career.

**For someone in normal eyes, he had everything - money, fame and a good life in terms of success the society had defined.** 

On his last instagram post, he wrote a small thought for his mother, mentioning of his eyes being blurry from tears of the past and his fleeing life, him negotiating between the two.

**On 14 June 2020, after 6 months of unspoken feelings of depression, he hanged himself in his Bandra home, in Mumbai, Maharashtra.
He was only 34 years old.**

I have a long time to yet discover if Heaven or Hell is what awaits at the end for us to eventually rest; but if they do, I hope his soul has Rested In Peace.
Thanks for reading.

### Acknowledgements

1. Table of contents hyperlinks - https://www.kaggle.com/sahidvelji/meet-the-grandmasters ([Sahid](https://www.kaggle.com/sahidvelji))
2. Map Labels - https://www.kaggle.com/sugandhkhobragade/caste-crimes-crimes-against-sc-2001-2013 ([sugandhkhobragade](https://www.kaggle.com/sugandhkhobragade))
3. Plotly/Geopandas Official documentations